# Text summarization - Cosine similarity

# Preparing the environment

In [1]:
import re
import nltk
import string
import numpy as np
import networkx as nx
from nltk.cluster.util import cosine_distance
from utils.visualize import visualize

nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\poseidon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\poseidon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def preprocess(text):
  formatted_text = text.lower()
  tokens = []
  for token in nltk.word_tokenize(formatted_text):
    tokens.append(token)
  tokens = [word for word in tokens if word not in stopwords and word not in string.punctuation]
  formatted_text = ' '.join(element for element in tokens)

  return formatted_text

In [3]:
original_text = """Artificial intelligence is human like intelligence. 
                   It is the study of intelligent artificial agents. 
                   Science and engineering to produce intelligent machines. 
                   Solve problems and have intelligence. 
                   Related to intelligent behavior. 
                   Developing of reasoning machines. 
                   Learn from mistakes and successes. 
                   Artificial intelligence is related to reasoning in everyday situations."""

original_text = re.sub(r'\s+', ' ', original_text)

original_text

'Artificial intelligence is human like intelligence. It is the study of intelligent artificial agents. Science and engineering to produce intelligent machines. Solve problems and have intelligence. Related to intelligent behavior. Developing of reasoning machines. Learn from mistakes and successes. Artificial intelligence is related to reasoning in everyday situations.'

# Function to calculate similarity between sentences

- Link: https://en.wikipedia.org/wiki/Cosine_similarity
- Step by step calculations: https://janav.wordpress.com/2013/10/27/tf-idf-and-cosine-similarity/

In [4]:
original_sentences = [sentence for sentence in nltk.sent_tokenize(original_text)]

formatted_sentences = [preprocess(original_sentence) for original_sentence in original_sentences]

In [5]:
def calculate_sentence_similarity(sentence1, sentence2):
  words1 = [word for word in nltk.word_tokenize(sentence1)]
  words2 = [word for word in nltk.word_tokenize(sentence2)]
  #print(words1)
  #print(words2)

  all_words = list(set(words1 + words2))
  #print(all_words)

  vector1 = [0] * len(all_words)
  vector2 = [0] * len(all_words)
  #print(vector1)
  #print(vector2)

  for word in words1: # Bag of words
    #print(word)
    vector1[all_words.index(word)] += 1
  for word in words2:
    vector2[all_words.index(word)] += 1
  
  #print(vector1)
  #print(vector2)

  return 1 - cosine_distance(vector1, vector2)

# Function to create the similarity matrix

In [6]:
# The higher the value, the greater the similarity between the sentences
# The more words in common, the greater the similarity
# 
def calculate_similarity_matrix(sentences):
  similarity_matrix = np.zeros((len(sentences), len(sentences)))
  #print(similarity_matrix)
  for i in range(len(sentences)):
    for j in range(len(sentences)):
      if i == j:
        continue
      similarity_matrix[i][j] = calculate_sentence_similarity(sentences[i], sentences[j])
  return similarity_matrix

# Function to summarize the texts

- Pagerank algorithm: https://en.wikipedia.org/wiki/PageRank


In [7]:
for i, score in enumerate(original_sentences):
  print(i, score)

0 Artificial intelligence is human like intelligence.
1 It is the study of intelligent artificial agents.
2 Science and engineering to produce intelligent machines.
3 Solve problems and have intelligence.
4 Related to intelligent behavior.
5 Developing of reasoning machines.
6 Learn from mistakes and successes.
7 Artificial intelligence is related to reasoning in everyday situations.


In [8]:
def summarize(text, number_of_sentences, percentage = 0):
  original_sentences = [sentence for sentence in nltk.sent_tokenize(text)]
  formatted_sentences = [preprocess(original_sentence) for original_sentence in original_sentences]
  similarity_matrix = calculate_similarity_matrix(formatted_sentences)
  #print(similarity_matrix)

  similarity_graph = nx.from_numpy_array(similarity_matrix)
  #print(similarity_graph.nodes)
  #print(similarity_graph.edges)

  scores = nx.pagerank(similarity_graph)
  #print(scores)
  ordered_scores = sorted(((scores[i], score) for i, score in enumerate(original_sentences)), reverse=True)
  #print(ordered_scores)

  if percentage > 0:
    number_of_sentences = int(len(formatted_sentences) * percentage)

  best_sentences = []
  for sentence in range(number_of_sentences):
    best_sentences.append(ordered_scores[sentence][1])
  
  return original_sentences, best_sentences, ordered_scores

In [9]:
original_sentences, best_sentences, scores = summarize(original_text, 3)

In [10]:
visualize('Artificial intelligence', original_sentences, best_sentences)

# Extracting texts from the Internet

In [11]:
from goose3.goose3 import Goose

g = Goose()
url = 'https://en.wikipedia.org/wiki/Automatic_summarization'
article = g.extract(url)

In [12]:
original_sentences, best_sentences, scores = summarize(article.cleaned_text, number_of_sentences=120, percentage=0.2)

In [13]:
visualize(article.title, original_sentences, best_sentences)